In [41]:
import requests
from bs4 import BeautifulSoup
from yahoofinancials import YahooFinancials
import yfinance
import pandas as pd
import datetime 
import concurrent.futures as cf
import multiprocessing
import vnstock
import json

In [70]:
tickers = list(vnstock.listing_companies(live=True)["ticker"])
stocks = set(tickers)

In [ ]:
vnstock.financial_ratio(tickers[0],"quarterly", "true").to_dict()

In [72]:
financial_ratio = {}
failed_stock = 0
for stock in stocks:
    try:
        financial_ratio[stock] = vnstock.financial_ratio(stock,"quarterly", "true").to_dict()
    except:
        failed_stock += 1
len(financial_ratio)

1249

In [73]:
with open("financial_ratio_data.json", "w") as f:
    json.dump(financial_ratio, f)

In [80]:
def filter_stock_buffet_way(financial_ratio):
    # Calculate metrics
    roe_ratio = {}
    eps_growth = {}
    result = []
    for stock, ratio_data in financial_ratio.items():
        stock_roe = 0
        stock_roe_down_count = 0
        stock_eps_growth = 0
        stock_eps_down_count = 0
        length = 0
        for quarter, quarter_data in ratio_data.items():
            roe = quarter_data["roe"] if not pd.isna(quarter_data["roe"]) else 0
            eps_change = quarter_data["epsChange"] if not pd.isna(quarter_data["epsChange"]) else 0
            if roe < 0: stock_roe_down_count += 1
            if eps_change < 0: stock_eps_down_count += 1
            stock_roe += roe
            stock_eps_growth += eps_change
            length += 1
        roe_ratio[stock] = {"avg": round(stock_roe / length, 2), "consistency": stock_roe_down_count}
        eps_growth[stock] = {"avg": round(stock_eps_growth / length, 2), "consistency": stock_eps_down_count}
    

    #Filter by conditions

    roe_threshold = 0.05
    eps_growth_threshold = 0.05
    
    for stock in roe_ratio:
        if roe_ratio[stock]["avg"] >= roe_threshold and eps_growth[stock]["avg"] >= eps_growth_threshold and roe_ratio[stock]["consistency"] == 0 and eps_growth[stock]["consistency"] < 2:
            result.append(stock)
            
    return result, roe_ratio, eps_growth
    
            

In [81]:
result, roe_ratio, eps_growth = filter_stock_buffet_way(financial_ratio)

In [82]:
result

['TED',
 'DAN',
 'TOT',
 'GSP',
 'M10',
 'DMN',
 'MBB',
 'SLS',
 'CMF',
 'RAL',
 'DBD',
 'FPT',
 'VCB',
 'GVT',
 'MSB',
 'DWS',
 'HNM',
 'DP1',
 'PTH',
 'CTR',
 'CMG',
 'CAP',
 'HDB']